In [1]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

# Множественная проверка гипотез

## Foodmart product sales 

In [3]:
sales = pd.read_csv('../../foodmart.sales.tsv', sep='\t', header=0, parse_dates=[2])

In [4]:
sales.head()

,product_id,store_id,date,sales
0,4,6,1997-01-01,4
1,25,6,1997-01-01,3
2,48,6,1997-01-01,3
3,76,6,1997-01-01,4
4,119,6,1997-01-01,3


In [5]:
sales.describe()

,product_id,store_id,sales
count,83391.000000,83391.000000,83391.000000
mean,785.412994,13.166529,3.199062
std,447.310605,6.243769,1.091561
min,1.000000,2.000000,1.000000
25%,406.000000,7.000000,3.000000
50%,789.000000,13.000000,3.000000
75%,1171.000000,17.000000,4.000000
max,1559.000000,24.000000,15.000000


In [6]:
products = pd.read_csv('../../foodmart.products.tsv', sep='\t', header=0)

In [7]:
products.head()

,product_class_id,product_id,brand_name,product_name,SKU,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
0,30,1,Washington,Washington Berry Juice,90748583674,2.85,8.39,6.39,False,False,30,14,16.9,12.60,7.40
1,52,2,Washington,Washington Mango Drink,96516502499,0.74,7.42,4.42,False,True,18,8,13.4,3.71,22.60
2,52,3,Washington,Washington Strawberry Drink,58427771925,0.83,13.10,11.10,True,True,17,13,14.4,11.00,7.77
3,19,4,Washington,Washington Cream Soda,64412155747,3.64,10.60,9.60,True,False,26,10,22.9,18.90,7.93
4,19,5,Washington,Washington Diet Soda,85561191439,2.19,6.66,4.65,True,False,7,10,20.7,21.90,19.20


In [8]:
products.describe()

,product_class_id,product_id,SKU,SRP,gross_weight,net_weight,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
count,1560.000000,1560.000000,1.560000e+03,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000
mean,54.403205,780.500000,5.562930e+10,2.117282,13.804821,11.791308,18.908333,9.475641,12.900692,12.915103,12.933750
std,30.872364,450.477524,2.580380e+10,0.932779,4.631014,4.693747,10.254237,2.638963,5.763655,5.844180,5.603036
min,1.000000,1.000000,1.003103e+10,0.500000,6.000000,3.050000,1.000000,5.000000,3.010000,3.000000,3.000000
25%,30.000000,390.750000,3.345056e+10,1.417500,9.657500,7.705000,10.000000,7.000000,7.860000,7.867500,8.300000
50%,57.500000,780.500000,5.629308e+10,2.130000,13.600000,11.600000,19.000000,9.000000,12.900000,12.700000,12.850000
75%,79.000000,1170.250000,7.798317e+10,2.792500,17.725000,16.000000,28.000000,12.000000,17.625000,18.200000,17.600000
max,110.000000,1560.000000,9.998368e+10,3.980000,21.900000,20.800000,36.000000,14.000000,22.900000,22.900000,22.900000


In [9]:
sales = sales.merge(products[['product_id', 'product_name']], 
                    on=['product_id'], how='inner')

In [10]:
sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


## Корреляция Пирсона

In [15]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                     columns=['product_name'], fill_value=0)

In [16]:
sparse_sales.head()

product_name         ADJ Rosy Sunglasses  Akron City Map  \
date       store_id                                        
1997-01-01 6                           0               0   
           14                          0               0   
1997-01-02 11                          0               0   
           23                          0               0   
1997-01-03 7                           0               0   

product_name         Akron Eyeglass Screwdriver  American Beef Bologna  \
date       store_id                                                      
1997-01-01 6                                  0                      0   
           14                                 0                      0   
1997-01-02 11                                 0                      0   
           23                                 0                      0   
1997-01-03 7                                  0                      0   

product_name         American Chicken Hot Dogs  American Cole Slaw  \
date       store_id                                                  
1997-01-01 6                                 0                   0   
           14                                0                   0   
1997-01-02 11                                0                   0   
           23                                0                   0   
1997-01-03 7                                 0                   0   

product_name         American Corned Beef  American Foot-Long Hot Dogs  \
date       store_id                                                      
1997-01-01 6                            0                            0   
           14                           0                            0   
1997-01-02 11                           0                            0   
           23                           0                            0   
1997-01-03 7                            0                            0   

product_name         American Low Fat Bologna  American Low Fat Cole Slaw  \
date       store_id                                                         
1997-01-01 6                                0                           0   
           14                               0                           0   
1997-01-02 11                               0                           0   
           23                               0                           0   
1997-01-03 7                                0                           0   

product_name         ...  Washington Apple Juice  Washington Berry Juice  \
date       store_id  ...                                                   
1997-01-01 6         ...                       0                       0   
           14        ...                       0                       0   
1997-01-02 11        ...                       0                       0   
           23        ...                       0                       0   
1997-01-03 7         ...                       0                       4   

product_name         Washington Cola  Washington Cranberry Juice  \
date       store_id                                                
1997-01-01 6                       0                           0   
           14                      0                           0   
1997-01-02 11                      0                           0   
           23                      0                           0   
1997-01-03 7                       0                           0   

product_name         Washington Cream Soda  Washington Diet Cola  \
date       store_id                                                
1997-01-01 6                             4                     0   
           14                            0                     0   
1997-01-02 11                            0                     0   
           23                            0                     0   
1997-01-03 7                             0                     0   

product_name         Wa

In [17]:
%%time 
corr_data = []

for i, lhs_column in enumerate(sparse_sales.columns):
    for j, rhs_column in enumerate(sparse_sales.columns):
        if i >= j:
            continue
        
        corr, p = pearsonr(sparse_sales[lhs_column], sparse_sales[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

Wall time: 2min 55s


In [19]:
sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns = ['product_A', 'product_B', 'corr', 'p']

In [20]:
sales_correlation.head()

,product_A,product_B,corr,p
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


Сколько гипотез об отсутствии корреляции отвергается без поправки на множественную проверку?

In [21]:
(sales_correlation.p < 0.05).value_counts()

False    982453
True     232008
Name: p, dtype: int64

## Поправка на множественную проверку

### Метод Холма

In [22]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha=0.05, 
                                            method='holm')

In [24]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [25]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


Посмотрим теперь, сколько гипотез об отсутствии корреляции отвергаются с учётом на множественную поправку Холма

In [26]:
sales_correlation.reject.value_counts()

False    1212733
True        1728
Name: reject, dtype: int64

In [27]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,3.265443e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,2.288034e-12,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,5.677558e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,1.966350e-11,True
...,...,...,...,...,...,...
662127,Ebony Potatos,Just Right Fancy Canned Anchovies,0.194914,4.073740e-08,4.940380e-02,True
565740,Cutting Edge Cole Slaw,Horatio Potato Chips,0.194875,4.099170e-08,4.971215e-02,True
615865,Discover Ravioli,High Quality Soft Napkins,0.194869,4.103075e-08,4.975947e-02,True
559150,Cormorant Toilet Bowl Cleaner,Plato Chunky Peanut Butter,0.194863,4.107241e-08,4.980995e-02,True


### Метод Бенджамини-Хохберга

In [28]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha=0.05, 
                                            method='fdr_bh') 

In [29]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [30]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,0.203716,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,0.956078,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,0.630699,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,0.600790,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,0.541916,False


In [31]:
sales_correlation.reject.value_counts()

False    1138407
True       76054
Name: reject, dtype: int64

In [32]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,1.632723e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,7.626793e-13,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,1.419393e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,3.932713e-12,True
...,...,...,...,...,...,...
108018,Best Choice Dried Dates,Tell Tale Party Nuts,0.105665,3.130716e-03,4.999516e-02,True
419799,Carrington Frozen Carrots,Sphinx Muffins,0.105664,3.130854e-03,4.999649e-02,True
1184578,Skinner Diet Cola,Special Wheat Puffs,0.105664,3.130881e-03,4.999649e-02,True
821999,Gauss Monthly Fashion Magazine,PigTail Beef TV Dinner,0.105664,3.130969e-03,4.999724e-02,True


Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора.

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [36]:
auc_data = pd.read_csv('../../AUCs.txt', sep='\t')

In [38]:
auc_data.rename({'Unnamed: 0': 'sets'}, axis=1, inplace=True)

In [39]:
auc_data.head()

,sets,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [43]:
from itertools import combinations
from scipy import stats

In [44]:
w_stat = pd.DataFrame(columns=['Model_1', 'Model_2', 'Wilcoxon', 'p-value'])
for k, (i, j) in enumerate(combinations([1, 2, 3, 4], 2)):
    w_stat.loc[k, 'Model_1'], w_stat.loc[k, 'Model_2'] = auc_data.columns[i], auc_data.columns[j]
    w_stat.loc[k, 'Wilcoxon'], w_stat.loc[k, 'p-value'] = stats.wilcoxon(auc_data.iloc[:, i], auc_data.iloc[:, j])

In [47]:
w_stat.sort_values(by='p-value')

,Model_1,Model_2,Wilcoxon,p-value
0,C4.5,C4.5+m,6.5,0.0107571
2,C4.5,C4.5+m+cf,11,0.0159064
5,C4.5+cf,C4.5+m+cf,10,0.0229091
3,C4.5+m,C4.5+cf,17,0.0463327
4,C4.5+m,C4.5+m+cf,22,0.327826
1,C4.5,C4.5+cf,43,0.861262


In [50]:
(w_stat['p-value'] < 0.05).value_counts()

True     4
False    2
Name: p-value, dtype: int64

In [52]:
auc_data.corr()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
C4.5,1.000000,0.991879,0.981828,0.982054
C4.5+m,0.991879,1.000000,0.991622,0.993424
C4.5+cf,0.981828,0.991622,1.000000,0.988264
C4.5+m+cf,0.982054,0.993424,0.988264,1.000000


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [53]:
reject, p_corrected, a1, a2 = multipletests(w_stat['p-value'], 
                                            alpha=0.05, 
                                            method='holm')

In [55]:
w_stat['p_corrected'] = p_corrected
(w_stat['p_corrected'] < 0.05).value_counts()

False    6
Name: p_corrected, dtype: int64

In [56]:
reject, p_corrected, a1, a2 = multipletests(w_stat['p-value'], 
                                            alpha=0.05, 
                                            method='fdr_bh')
w_stat['p_corrected'] = p_corrected
(w_stat['p_corrected'] < 0.05).value_counts()

True     3
False    3
Name: p_corrected, dtype: int64